In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [3]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [4]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [5]:
df_train.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,,
0001_01,False,39.0,False,0.00000,0.000000,0.000000,0.000000,0.000000,False,True,1,0,0,1,1,0,0,0,0,0
0002_01,False,24.0,False,4.70048,2.302585,3.258097,6.309918,3.806662,True,False,0,0,0,1,0,0,0,0,1,0


In [6]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV

## Tuning

In [23]:
params = {
    'n_neighbors' : [12,13,14],
    'weights' : ['uniform','distance'],
    'metric' : ['minkowski','euclidean','manhattan']
}

In [24]:
grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=KFold(5, shuffle=True, random_state=42),
    verbose=3,
    return_train_score=True
)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END metric=minkowski, n_neighbors=12, weights=uniform;, score=(train=0.797, test=0.761) total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=12, weights=uniform;, score=(train=0.798, test=0.759) total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=12, weights=uniform;, score=(train=0.801, test=0.768) total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=12, weights=uniform;, score=(train=0.799, test=0.775) total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=12, weights=uniform;, score=(train=0.798, test=0.765) total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=12, weights=distance;, score=(train=0.935, test=0.760) total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=12, weights=distance;, score=(train=0.937, test=0.756) total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=12, weights=distance;, score=(train=0.935, test=0.768) total time=   0.0s
[CV 4/5]

[CV 1/5] END metric=manhattan, n_neighbors=13, weights=uniform;, score=(train=0.802, test=0.779) total time=   0.0s
[CV 2/5] END metric=manhattan, n_neighbors=13, weights=uniform;, score=(train=0.805, test=0.775) total time=   0.0s
[CV 3/5] END metric=manhattan, n_neighbors=13, weights=uniform;, score=(train=0.810, test=0.774) total time=   0.0s
[CV 4/5] END metric=manhattan, n_neighbors=13, weights=uniform;, score=(train=0.811, test=0.781) total time=   0.0s
[CV 5/5] END metric=manhattan, n_neighbors=13, weights=uniform;, score=(train=0.808, test=0.779) total time=   0.0s
[CV 1/5] END metric=manhattan, n_neighbors=13, weights=distance;, score=(train=0.936, test=0.770) total time=   0.0s
[CV 2/5] END metric=manhattan, n_neighbors=13, weights=distance;, score=(train=0.938, test=0.767) total time=   0.0s
[CV 3/5] END metric=manhattan, n_neighbors=13, weights=distance;, score=(train=0.936, test=0.773) total time=   0.0s
[CV 4/5] END metric=manhattan, n_neighbors=13, weights=distance;, sco

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [12, 13, 14],
                         'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='accuracy', verbose=3)

In [25]:
grid.best_params_

{'metric': 'manhattan', 'n_neighbors': 13, 'weights': 'uniform'}

In [26]:
#df_grid = pd.DataFrame(grid.cv_results_)
df_grid.groupby('param_weights')['mean_test_score'].mean()

param_weights
distance    0.754946
uniform     0.758634
Name: mean_test_score, dtype: float64

In [27]:
pd.DataFrame(grid.cv_results_)[['mean_train_score','std_train_score','mean_test_score','std_test_score']].loc[grid.best_index_]

mean_train_score    0.807345
std_train_score     0.003264
mean_test_score     0.777523
std_test_score      0.002534
Name: 14, dtype: float64

In [44]:
model = grid.best_estimator_
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=5, n_estimators=200)

## Cross Validation

# Submission

In [51]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,True,0,0,0,1,0,0,0,0,0,1
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,False,0,0,0,1,0,0,0,0,1,0


In [52]:
y_test = model.predict(X_test)

In [53]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_rnforest_2022-11-02.csv')